In [1]:
import pandas as pd
import numpy as np

In [2]:
from pyspark.sql import SparkSession
#Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Liveability")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)


your 131072x1 screen size is bogus. expect trouble
24/10/06 15:21:13 WARN Utils: Your hostname, DESKTOP-Q5SP5SI resolves to a loopback address: 127.0.1.1; using 172.20.36.110 instead (on interface eth0)
24/10/06 15:21:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/06 15:21:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/06 15:21:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/06 15:21:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
df = spark.read.parquet('../data/landing0/combined_data_with_geolocation.parquet', header=True, inferSchema=True)
df.show()

+--------+------------+---------------+--------+----------+---------+----------------+
|Postcode|School_Count|groceries_Count|hc_Count|  Latitude|Longitude|   Postcode_Name|
+--------+------------+---------------+--------+----------+---------+----------------+
|    3000|          24|             42|       8|-37.815205|144.96394|       Melbourne|
|    3002|          14|             17|      10|-37.816143|144.98045|  East Melbourne|
|    3003|           7|             14|       2| -37.81145| 144.9254|  West Melbourne|
|    3004|           7|             12|       2| -37.83016|144.98045|       Melbourne|
|    3006|          11|             23|       6|-37.824547|144.96394|     South Wharf|
|    3008|           3|              3|       1|-37.817066|144.94191|       Docklands|
|    3010|           1|              1|    NULL|-37.798447| 144.9621|       Parkville|
|    3011|          31|             49|      14| -37.79602|144.90063|       Footscray|
|    3012|           9|             14|    

In [11]:
RAI_df = spark.read.parquet('../data/landing/RAI.parquet')

In [12]:
print((RAI_df))

+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------+-----------------+
|             suburbs|           f1B_RAI|           f2B_RAI|           f3B_RAI|           h2B_RAI|           h3B_RAI|           h4B_RAI|           all_RAI|postcode|__index_level_0__|
+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------+-----------------+
|Albert Park-Middl...| 259.7647058823529|             184.0|128.37209302325581|149.59349593495935| 96.84210526315789| 74.84745762711864|169.84615384615384|    3054|                2|
|            Armadale|256.74418604651163|193.68421052631578|122.66666666666666|142.45161290322582|105.14285714285714| 83.32075471698113|197.14285714285717|    3052|                3|
|       Carlton North|             276.0|200.72727272727272|157.71428571428572|161.16

In [7]:
df = df.withColumnRenamed("School_Count", "Number of Schools")
df = df.withColumnRenamed("groceries_Count", "Number of Groceries")
df = df.withColumnRenamed("hc_Count", "Number of Healthcare Services")
df = df.withColumnRenamed("Postcode_Name", "Suburbs")
df

Postcode,Number of Schools,Number of Groceries,Number of Healthcare Services,Latitude,Longitude,Suburbs
3000,24,42,8,-37.815205,144.96394,Melbourne
3002,14,17,10,-37.816143,144.98045,East Melbourne
3003,7,14,2,-37.81145,144.9254,West Melbourne
3004,7,12,2,-37.83016,144.98045,Melbourne
3006,11,23,6,-37.824547,144.96394,South Wharf
3008,3,3,1,-37.817066,144.94191,Docklands
3010,1,1,NULL,-37.798447,144.9621,Parkville
3011,31,49,14,-37.79602,144.90063,Footscray
3012,9,14,5,-37.814625,144.84563,Tottenham
3013,14,21,3,-37.819813,144.88138,Yarraville


In [35]:

output = df.join(df1,['Postcode'],"inner")
output.show()

+--------+-----------------+-------------------+-----------------------------+----------+---------+----------------+--------------------------------+-------------------------------+
|Postcode|Number of Schools|Number of Groceries|Number of Healthcare Services|  Latitude|Longitude|         Suburbs|1-2 Bedder Affordability Metrics|3+ Bedder Affordability Metrics|
+--------+-----------------+-------------------+-----------------------------+----------+---------+----------------+--------------------------------+-------------------------------+
|    3000|               24|                 42|                            8|-37.815205|144.96394|       Melbourne|                4.65482052280358|              10.16330811857669|
|    3002|               14|                 17|                           10|-37.816143|144.98045|  East Melbourne|               4.768741414266279|             10.159587087927141|
|    3006|               11|                 23|                            6|-37.824547|1